# クエリを分割しログを眺めてみる

Knowledge bases for Amazon Bedrock では、クエリをいくつかのサブクエリに分割し各クエリで RAG へ検索する機能が提供されています。

例えば、次の問い合わせがあったとします。

```text
給与口座を変更したいです。いつまでに何をすればいいですか？
```

この問いに対して、以下のように細かい問い合わせに分割することが`クエリ分割`です。


```markdown
- 給与口座を変更するにはいつまでに何をすればいいですか?
- 給与口座変更の手続き方法
- 給与口座変更に必要な書類
- 給与口座変更の期限
```

クエリ分割によって RAG へ問い合わせたい内容が明確化され、具体的で詳細な情報を効率的に取得できます。

## クエリ分割の実装

クエリ分割の実装は RetrieveAndGenerate API の場合、  `orchestrationConfiguration` から利用可能です。このセクションではクエリ分割を行ったのちに、 Bedrock の 実行ログを確認しクエリ分割がどのように行われているかをみてみましょう。

### boto3 の更新

執筆時点での boto3 のバージョンは 1.34.131 だったため、 boto3 の更新を行います。

```log
sagemaker-user@default:~/DevelopersIO-2024-bedrock/terraform/rag$ pip show boto3
Name: boto3
Version: 1.34.131
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: amazon-q-developer-jupyterlab-ext, amazon-sagemaker-sql-editor, amazon-sagemaker-sql-execution, amazon_sagemaker_jupyter_ai_q_developer, autogluon.common, autogluon.core, autogluon.multimodal, aws-glue-sessions, langchain-aws, PyAthena, redshift_connector, sagemaker, sagemaker-studio-analytics-extension, sagemaker-studio-sparkmagic-lib
```

以下コマンドで boto3 の更新を行います。

#### コピペ用

In [ ]:
pip install -U boto3==1.34.144

※ 依存関係で怒られますが、ハンズオンの本質ではないためスキップします。

In [ ]:
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")
prompt = "オフィスの会議室を借りたいです。何時から借りれて、どのように予約すればいいですか？"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "orchestrationConfiguration": {
                "queryTransformationConfiguration": {"type": "QUERY_DECOMPOSITION"}
            },
        },
    },
)

print(response["output"]["text"])

## ログを確認してみる

それでは、ログを確認してクエリ分割がどのように行われているのかみてみましょう。

今回は、 LLM の実行ログ `/aws/bedrock/devio-2024-invoke` を確認してみます。

### クエリの書き換え

まずはじめに RAG に対してクエリの書き換え及び、分割が行われます。システムプロンプトは以下のとおりです。

```text
You are a query generation agent. Given the conversation history (optional) and a user question, your task is to determine the optimal queries.

You should consider the following two steps:
Step 1. If conversation history is available, decide if the user question misses any necessary context. If so, rewrite the user question to be a standalone question that captures the relevant context from the given conversation history.
Step 2. Decide if the standalone question is a complex question. If so, decompose it into a set of relevant queries, which can be easier to answer.

Please pay attention to the conversation history. If the user question is already a standalone question, you can skip Step 1.
If the standalone question does not require a further decomposition, you can skip Step 2.

Here’s an example for Step 1:

<example>
Input:
<conversation_history>
<conversation>
<question>How many vehicles can I include in a quote in Kansas</question>
<answer>You can include 5 vehicles in a quote if you live in Kansas</answer>
</conversation>
</conversation_history>

User: What about texas?

Output:
<generated_queries><standalone_question>How many vehicles can be included in a quote in Texas?<standalone_question></generated_queries>
</example>

Here are two examples for Step 2:

<example>
Input:

User: Were both of the following rock groups formed in California: Dig and Thinking Fellers Union Local 282?

Output:
<generated_queries><standalone_question>Were both of the following rock groups formed in California: Dig and Thinking Fellers Union Local 282?</standalone_question><query>What is the formation place of rock group Dig?</query><query>What is the formation place of the rock group Thinking Fellers Union Local 282?</query></generated_queries>
</example>

<example>
Input:

User: in 2018 , what percentage of undeveloped acres were located in the u.s?

Output:
<generated_queries><standalone_question>in 2018 , what percentage of undeveloped acres were located in the u.s?</standalone_question><query>the net undeveloped acres in 2018 of u.s.</query><query>the total net undeveloped acres in 2018</query></generated_queries>
</example>

Here is the current conversation history:
<conversation_history>

</conversation_history>

Please output your response between <generated_queries> </generated_queries> tags, the standalone question within <standalone_question> </standalone_question> and each decomposed query within <query> </query> tags.
```

今回の場合だと、「**オフィスの会議室を借りたいです。何時から借りれて、どのように予約すればいいですか？**」は次のように分割されました。

- 書き換えられたユーザークエリ：オフィスの会議室を借りるための手順と予約可能な時間帯を教えてください。
- サブクエリ1：会議室の予約方法
- サブクエリ2：会議室の利用可能時間

```json
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T12:09:57Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-sagemaker-domain/SageMaker"
    },
    "region": "us-west-2",
    "requestId": "bebe8544-18ac-4654-a17c-b0dda1e4678d",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "オフィスの会議室を借りたいです。何時から借りれて、どのように予約すればいいですか？"
                        }
                    ]
                }
            ],
            "system": "You are a query generation agent. Given the conversation history (optional) and a user question, your task is to determine the optimal queries.

You should consider the following two steps:
Step 1. If conversation history is available, decide if the user question misses any necessary context. If so, rewrite the user question to be a standalone question that captures the relevant context from the given conversation history.
Step 2. Decide if the standalone question is a complex question. If so, decompose it into a set of relevant queries, which can be easier to answer.

Please pay attention to the conversation history. If the user question is already a standalone question, you can skip Step 1.
If the standalone question does not require a further decomposition, you can skip Step 2.

Here’s an example for Step 1:

<example>
Input:
<conversation_history>
<conversation>
<question>How many vehicles can I include in a quote in Kansas</question>
<answer>You can include 5 vehicles in a quote if you live in Kansas</answer>
</conversation>
</conversation_history>

User: What about texas?

Output:
<generated_queries><standalone_question>How many vehicles can be included in a quote in Texas?<standalone_question></generated_queries>
</example>

Here are two examples for Step 2:

<example>
Input:

User: Were both of the following rock groups formed in California: Dig and Thinking Fellers Union Local 282?

Output:
<generated_queries><standalone_question>Were both of the following rock groups formed in California: Dig and Thinking Fellers Union Local 282?</standalone_question><query>What is the formation place of rock group Dig?</query><query>What is the formation place of the rock group Thinking Fellers Union Local 282?</query></generated_queries>
</example>

<example>
Input:

User: in 2018 , what percentage of undeveloped acres were located in the u.s?

Output:
<generated_queries><standalone_question>in 2018 , what percentage of undeveloped acres were located in the u.s?</standalone_question><query>the net undeveloped acres in 2018 of u.s.</query><query>the total net undeveloped acres in 2018</query></generated_queries>
</example>

Here is the current conversation history:
<conversation_history>

</conversation_history>

Please output your response between <generated_queries> </generated_queries> tags, the standalone question within <standalone_question> </standalone_question> and each decomposed query within <query> </query> tags.
",
            "max_tokens": 2048,
            "temperature": 0,
            "top_p": 1,
            "stop_sequences": [
                "
Observation"
            ],
            "top_k": 50
        },
        "inputTokenCount": 643
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "id": "msg_bdrk_01PhsAF59MbyM2QYrWuL3ZQj",
            "type": "message",
            "role": "assistant",
            "model": "claude-3-sonnet-20240229",
            "content": [
                {
                    "type": "text",
                    "text": "<generated_queries>
<standalone_question>オフィスの会議室を借りるための手順と予約可能な時間帯を教えてください。</standalone_question>
<query>会議室の予約方法</query>
<query>会議室の利用可能時間</query>
</generated_queries>"
                }
            ],
            "stop_reason": "end_turn",
            "stop_sequence": null,
            "usage": {
                "input_tokens": 643,
                "output_tokens": 88
            }
        },
        "outputTokenCount": 88
    }
}
```

## エンべディング

書き換えられた、分割されたクエリはエンべディングされることがわかります。

```json
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T12:09:59Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-kb-role/BedrockQueryKnowledgeBaseSession-DGPJXQCIT2"
    },
    "region": "us-west-2",
    "requestId": "d910f18b-feca-4688-a551-16271770b88d",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "オフィスの会議室を借りるための手順と予約可能な時間帯を教えてください。"
        },
        "inputTokenCount": 15
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "embedding": [
                0.9453125,
            ],
            "inputTextTokenCount": 15
        }
    }
}
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T12:09:59Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-kb-role/BedrockQueryKnowledgeBaseSession-DGPJXQCIT2"
    },
    "region": "us-west-2",
    "requestId": "790da2d5-5524-48ac-9582-deef37e8001b",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "会議室の予約方法"
        },
        "inputTokenCount": 5
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "embedding": [
                0.89453125,
            ],
            "inputTextTokenCount": 5
        }
    }
}
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T12:09:59Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-kb-role/BedrockQueryKnowledgeBaseSession-DGPJXQCIT2"
    },
    "region": "us-west-2",
    "requestId": "93db22ac-103b-4c68-91a3-c02c354e0968",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "会議室の利用可能時間"
        },
        "inputTokenCount": 5
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "embedding": [
                0.8125,
            ],
            "inputTextTokenCount": 5
        }
    }
}
```

## ベクトル検索及び回答生成

各クエリに対してベクトル検索が行われ回答が生成されます。プロンプトは以下のとおりです。

分割されたクエリは 3 つであることに対し、5つのソースが結果として返ってきているのは何か仕組みがありそうですね。後ほどブログにできればと思います。

```text
You are a question answering agent. I will provide you with a set of search results. The user will provide you with a question. Your job is to answer the user's question using only information from the search results. If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.

Here are the search results in numbered order:
<search_results>

<search_result>
<content>
# オンボーディングプラン  ## 1. 入社前準備（-2 週間〜入社日）  ### 1.1 人事部門の準備  - [ ] 雇用契約書の準備と送付 - [ ] 必要な機器（ノート PC、スマートフォン等）の手配 - [ ] 社内システムのアカウント作成  ### 1.2 営業部門の準備  - [ ] オンボーディングスケジュールの作成 - [ ] メンターの指名 - [ ] 初期の顧客リストの準備  ### 1.3 IT 部門の準備  - [ ] CRM システムのアクセス権設定 - [ ] 営業支援ツールのライセンス確保 - [ ] 社内ネットワークへのアクセス設定  ## 2. 入社初日  ### 2.1 オリエンテーション（午前）  - [ ] 会社概要の説明（歴史、ミッション、ビジョン、バリュー） - [ ] オフィスツアー - [ ] チームメンバーの紹介  ### 2.2 システムセットアップ（午後）  - [ ] 社用 PC とスマートフォンのセットアップ - [ ] 各種アカウントのアクティベーション - [ ] CRM システムへのログイン確認  ## 3. 入社 1 週間目  ### 3.1 会社と製品の理解  - [ ] 会社の歴史と成長戦略の詳細レビュー - [ ] 製品・サービスラインナップの詳細学習   - 各製品の特徴、ターゲット顧客、価格帯   - 製品デモンストレーションの見学 - [ ] 業界動向と市場ポジショニングの理解  ### 3.2 営業プロセスの概要  - [ ] 営業サイクルの各段階の説明 - [ ] リード獲得から成約までのフロー理解 - [ ] CRM システムの基本操作トレーニング  ### 3.3 チーム文化の理解  - [ ] 朝会への参加 - [ ] 週次営業ミーティングへの参加 - [ ] メンターとの初回 1on1 セッション  ## 4.
</content>
<source>
1
</source>
</search_result>
<search_result>
<content>
継続的な成長支援  ### 6.1 定期的なスキル評価  - [ ] 月次の営業スキル評価 - [ ] 四半期ごとの目標達成度レビュー - [ ] 年 2 回の総合パフォーマンス評価  ### 6.2 ongoing トレーニング  - [ ] 週 1 回の商品知識アップデートセッション - [ ] 月 1 回の営業スキル向上ワークショップ - [ ] 四半期ごとの業界動向ブリーフィング  ### 6.3 キャリア開発サポート  - [ ] 個人開発計画の作成（入社 3 ヶ月後） - [ ] キャリアパス説明会への参加 - [ ] リーダーシップ開発プログラムの紹介  ## 7. 評価とフィードバック  ### 7.1 定期的なフィードバックセッション  - [ ] 週 1 回のメンターとの 1on1 ミーティング - [ ] 月 1 回の上司との進捗確認ミーティング - [ ] 四半期ごとの 360 度フィードバック  ### 7.2 成果の可視化  - [ ] 日次の活動報告の習慣化 - [ ] 週次の成果レポート作成 - [ ] 月次の成績ランキングへの参加  ### 7.3 改善計画  - [ ] パフォーマンス改善が必要な場合の個別計画作成 - [ ] スキルギャップに対する追加トレーニングの実施 - [ ] メンターによる集中的なサポート期間の設定  ## 8. チーム統合と文化醸成  ### 8.1 チームビルディング活動  - [ ] 月 1 回のチーム懇親会への参加 - [ ] 四半期ごとのチーム外活動（ボランティアなど） - [ ] 年 1 回の営業部門リトリートへの参加  ### 8.2 知識共有の促進  - [ ] 週 1 回のベストプラクティス共有セッション - [ ] 月 1 回の成功事例プレゼンテーション - [ ] 社内ナレッジベースへの定期的な貢献  ### 8.3 モチベーション向上策  - [ ] 月間 MVP 表彰への参加 - [ ] 四半期ごとのインセンティブプログラムの説明 - [ ] 年間トップセールス表彰式への参加
</content>
<source>
2
</source>
</search_result>
<search_result>
<content>
# 従業員住所変更手続き：人事君システム利用ガイド  ## 1. 概要  本ガイドは、社員が転居する際の「人事君」システムを使用した社内手続きを説明します。  ## 2. 転居前の手続き  ### 2.1 人事君システムでの事前報告  - 転居の 14 日前までに、「人事君」システムで報告 - 手順：人事君ログイン > 「住所変更申請」フォーム入力  ### 2.2 人事君システムでの書類準備  「人事君」システムから以下の書類をダウンロード・準備：  - 住所変更届（人事君指定様式） - 新住所証明書アップロード用フォーム  ## 3. 転居後の手続き  ### 3.1 人事君システムでの住所変更確定  - 転居後 7 日以内に、「人事君」システムで住所変更を確定 - 手順：人事君ログイン > 「住所変更確定」ページ > 書類アップロード  ### 3.2 人事君データベースの更新  - 「人事君」個人情報ページで住所を直接更新 - システム部が人事君データベースの更新を確認  ### 3.3 人事君での社会保険情報更新  - 健康保険証の住所変更：人事君の「保険情報更新」ページで申請 - 国民年金の住所変更：個人で実施（人事君対象外）  ## 4. テレワーク関連の人事君更新  ### 4.1 人事君でのネットワーク設定申請  - 自宅テレワーク用ネットワーク設定変更を人事君で申請 - 手順：人事君 > 「IT 支援」タブ > 「ネットワーク設定変更」フォーム  ### 4.2 人事君での勤務地情報更新  - 新住所をテレワーク可能場所として人事君に登録 - 上司承認ワークフローは人事君システム内で自動実行  ## 5. その他の人事君関連更新事項  - 通勤経路変更：人事君の「通勤手当再計算」ページで申請 - 緊急連絡先変更：人事君の個人情報ページで直接更新  ## 6. 人事君システムに関する問い合わせ先  - 人事部（人事君管理者）：hr-jinji-kun@example.com - IT 部（人事君技術サポート）：jinji-kun-support@example.com - 総務部（人事君データ管理）：jinji-kun-data@example.com  人事君システムの操作方法など、不明点はこちらにお問い合わせください。
</content>
<source>
3
</source>
</search_result>
<search_result>
<content>
入社 2〜4 週間目  ### 4.1 詳細な製品トレーニング  - [ ] 各製品・サービスの技術的詳細の学習 - [ ] 製品利用事例とケーススタディの理解 - [ ] 製品デモンストレーションの実践  ### 4.2 営業スキルトレーニング  - [ ] 効果的な電話営業テクニックの学習 - [ ] 商談ロールプレイングセッション - [ ] 提案書作成ワークショップ  ### 4.3 顧客管理実践  - [ ] CRM システムでの顧客情報管理演習 - [ ] リードスコアリングの方法論理解 - [ ] 顧客セグメンテーション演習  ### 4.4 コンプライアンストレーニング  - [ ] 営業倫理に関するオンラインコース受講 - [ ] 個人情報保護法の理解 - [ ] 反社会的勢力対応マニュアルの確認  ## 5. 入社 1〜3 ヶ月目  ### 5.1 実践的営業活動  - [ ] メンター同行での顧客訪問（週 1-2 回） - [ ] 電話でのリードフォロー（日々の業務） - [ ] 小規模案件の単独担当  ### 5.2 提案・見積スキルの向上  - [ ] 提案書作成演習（週 1 回） - [ ] 見積作成システムの詳細トレーニング - [ ] プレゼンテーションスキル向上ワークショップ  ### 5.3 業界知識の深化  - [ ] 業界専門誌の定期購読開始 - [ ] 競合他社の製品比較分析 - [ ] 業界セミナーやウェビナーへの参加  ### 5.4 社内ネットワーキング  - [ ] 他部門（製品開発、マーケティング等）との交流会 - [ ] 経営陣との懇談会 - [ ] 社内メンターシッププログラムへの参加  ## 6.
</content>
<source>
4
</source>
</search_result>
<search_result>
<content>
# オフィス利用ガイドライン  1. 一般的なルール     - オフィスの開閉時間を遵守してください（例：平日 8:00-20:00）    - 身分証明書やセキュリティカードを常に携帯し、必要に応じて提示してください    - オフィス内では適切な服装を心がけてください    - 喫煙は指定された場所でのみ行ってください  2. セキュリティ     - 部外者を無断でオフィスに入れないでください    - 機密情報の取り扱いには十分注意し、不要な書類は適切に廃棄してください    - パソコンから離れる際は必ずロックしてください    - セキュリティに関する懸念事項は速やかに報告してください  3. 共有スペース     - 会議室やミーティングスペースは予約システムを利用し、使用後は整理整頓してください    - キッチンや休憩スペースは清潔に保ち、使用後は片付けてください    - 騒音に配慮し、他の従業員の作業を妨げないようにしてください  4. 設備・備品の利用     - オフィス設備や備品は丁寧に扱い、故障や不具合は速やかに報告してください    - 個人的な目的での過度な設備利用（プリンター等）は控えてください    - エネルギー節約のため、不要な照明や機器の電源はこまめに切ってください  5. 健康・安全     - オフィス内での事故や怪我は直ちに報告してください    - 緊急時の避難経路と集合場所を把握しておいてください    - 体調不良時は無理せず、適切な対応（在宅勤務、休暇取得等）を取ってください  6. コミュニケーション     - 同僚への敬意を持ち、ハラスメントのない職場環境を維持してください    - オープンスペースでの長時間の私的な会話は控えてください    - 重要な連絡事項は適切な方法（メール、掲示板等）で共有してください  7. リモートワーク     - リモートワーク時も、通常のオフィスワーク同様のプロ意識を持って業務に取り組んでください    - オンライン会議の際は、適切な環境（背景、照明、音声）を整えてください  8. 環境への配慮     - ゴミの分別を徹底し、リサイクルに協力してください    - 紙の使用を最小限に抑え、可能な限りデジタル化を推進してください  9. 来客対応     - 来客の際は礼儀正しく対応し、必要に応じて適切な案内をしてください    - 機密情報が見える場所に来客を案内する際は十分注意してください  10. ガイドラインの遵守     - このガイドラインに違反する行為を見かけた場合は、適切な方法で報告してください     - ガイドラインは定期的に見直され、更新される可能性があります。最新版を常に確認してください  このガイドラインは、快適で生産的なオフィス環境を維持するためのものです。皆様のご協力をお願いいたします。
</content>
<source>
5
</source>
</search_result>


</search_results>

If you reference information from a search result within your answer, you must include a citation to source where the information was found. Each result has a corresponding source ID that you should reference.

Note that <sources> may contain multiple <source> if you include information from multiple results in your answer.

Do NOT directly quote the <search_results> in your answer. Your job is to answer the user's question as concisely as possible.

You must output your answer in the following format. Pay attention and follow the formatting and spacing exactly:
<answer>
<answer_part>
<text>
first answer text
</text>
<sources>
<source>source ID</source>
</sources>
</answer_part>
<answer_part>
<text>
second answer text
</text>
<sources>
<source>source ID</source>
</sources>
</answer_part>
</answer>
```

実際のログは以下のとおりです。

```json
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T12:09:59Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-sagemaker-domain/SageMaker"
    },
    "region": "us-west-2",
    "requestId": "bb5948e1-ff4b-4984-952d-9acc93bbd25c",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "オフィスの会議室を借りるための手順と予約可能な時間帯を教えてください。"
                        }
                    ]
                }
            ],
            "system": "You are a question answering agent. I will provide you with a set of search results. The user will provide you with a question. Your job is to answer the user's question using only information from the search results. If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.\n\nHere are the search results in numbered order:\n<search_results>\n\n<search_result>\n<content>\n# オンボーディングプラン  ## 1. 入社前準備（-2 週間〜入社日）  ### 1.1 人事部門の準備  - [ ] 雇用契約書の準備と送付 - [ ] 必要な機器（ノート PC、スマートフォン等）の手配 - [ ] 社内システムのアカウント作成  ### 1.2 営業部門の準備  - [ ] オンボーディングスケジュールの作成 - [ ] メンターの指名 - [ ] 初期の顧客リストの準備  ### 1.3 IT 部門の準備  - [ ] CRM システムのアクセス権設定 - [ ] 営業支援ツールのライセンス確保 - [ ] 社内ネットワークへのアクセス設定  ## 2. 入社初日  ### 2.1 オリエンテーション（午前）  - [ ] 会社概要の説明（歴史、ミッション、ビジョン、バリュー） - [ ] オフィスツアー - [ ] チームメンバーの紹介  ### 2.2 システムセットアップ（午後）  - [ ] 社用 PC とスマートフォンのセットアップ - [ ] 各種アカウントのアクティベーション - [ ] CRM システムへのログイン確認  ## 3. 入社 1 週間目  ### 3.1 会社と製品の理解  - [ ] 会社の歴史と成長戦略の詳細レビュー - [ ] 製品・サービスラインナップの詳細学習   - 各製品の特徴、ターゲット顧客、価格帯   - 製品デモンストレーションの見学 - [ ] 業界動向と市場ポジショニングの理解  ### 3.2 営業プロセスの概要  - [ ] 営業サイクルの各段階の説明 - [ ] リード獲得から成約までのフロー理解 - [ ] CRM システムの基本操作トレーニング  ### 3.3 チーム文化の理解  - [ ] 朝会への参加 - [ ] 週次営業ミーティングへの参加 - [ ] メンターとの初回 1on1 セッション  ## 4.\n</content>\n<source>\n1\n</source>\n</search_result>\n<search_result>\n<content>\n継続的な成長支援  ### 6.1 定期的なスキル評価  - [ ] 月次の営業スキル評価 - [ ] 四半期ごとの目標達成度レビュー - [ ] 年 2 回の総合パフォーマンス評価  ### 6.2 ongoing トレーニング  - [ ] 週 1 回の商品知識アップデートセッション - [ ] 月 1 回の営業スキル向上ワークショップ - [ ] 四半期ごとの業界動向ブリーフィング  ### 6.3 キャリア開発サポート  - [ ] 個人開発計画の作成（入社 3 ヶ月後） - [ ] キャリアパス説明会への参加 - [ ] リーダーシップ開発プログラムの紹介  ## 7. 評価とフィードバック  ### 7.1 定期的なフィードバックセッション  - [ ] 週 1 回のメンターとの 1on1 ミーティング - [ ] 月 1 回の上司との進捗確認ミーティング - [ ] 四半期ごとの 360 度フィードバック  ### 7.2 成果の可視化  - [ ] 日次の活動報告の習慣化 - [ ] 週次の成果レポート作成 - [ ] 月次の成績ランキングへの参加  ### 7.3 改善計画  - [ ] パフォーマンス改善が必要な場合の個別計画作成 - [ ] スキルギャップに対する追加トレーニングの実施 - [ ] メンターによる集中的なサポート期間の設定  ## 8. チーム統合と文化醸成  ### 8.1 チームビルディング活動  - [ ] 月 1 回のチーム懇親会への参加 - [ ] 四半期ごとのチーム外活動（ボランティアなど） - [ ] 年 1 回の営業部門リトリートへの参加  ### 8.2 知識共有の促進  - [ ] 週 1 回のベストプラクティス共有セッション - [ ] 月 1 回の成功事例プレゼンテーション - [ ] 社内ナレッジベースへの定期的な貢献  ### 8.3 モチベーション向上策  - [ ] 月間 MVP 表彰への参加 - [ ] 四半期ごとのインセンティブプログラムの説明 - [ ] 年間トップセールス表彰式への参加\n</content>\n<source>\n2\n</source>\n</search_result>\n<search_result>\n<content>\n# 従業員住所変更手続き：人事君システム利用ガイド  ## 1. 概要  本ガイドは、社員が転居する際の「人事君」システムを使用した社内手続きを説明します。  ## 2. 転居前の手続き  ### 2.1 人事君システムでの事前報告  - 転居の 14 日前までに、「人事君」システムで報告 - 手順：人事君ログイン > 「住所変更申請」フォーム入力  ### 2.2 人事君システムでの書類準備  「人事君」システムから以下の書類をダウンロード・準備：  - 住所変更届（人事君指定様式） - 新住所証明書アップロード用フォーム  ## 3. 転居後の手続き  ### 3.1 人事君システムでの住所変更確定  - 転居後 7 日以内に、「人事君」システムで住所変更を確定 - 手順：人事君ログイン > 「住所変更確定」ページ > 書類アップロード  ### 3.2 人事君データベースの更新  - 「人事君」個人情報ページで住所を直接更新 - システム部が人事君データベースの更新を確認  ### 3.3 人事君での社会保険情報更新  - 健康保険証の住所変更：人事君の「保険情報更新」ページで申請 - 国民年金の住所変更：個人で実施（人事君対象外）  ## 4. テレワーク関連の人事君更新  ### 4.1 人事君でのネットワーク設定申請  - 自宅テレワーク用ネットワーク設定変更を人事君で申請 - 手順：人事君 > 「IT 支援」タブ > 「ネットワーク設定変更」フォーム  ### 4.2 人事君での勤務地情報更新  - 新住所をテレワーク可能場所として人事君に登録 - 上司承認ワークフローは人事君システム内で自動実行  ## 5. その他の人事君関連更新事項  - 通勤経路変更：人事君の「通勤手当再計算」ページで申請 - 緊急連絡先変更：人事君の個人情報ページで直接更新  ## 6. 人事君システムに関する問い合わせ先  - 人事部（人事君管理者）：hr-jinji-kun@example.com - IT 部（人事君技術サポート）：jinji-kun-support@example.com - 総務部（人事君データ管理）：jinji-kun-data@example.com  人事君システムの操作方法など、不明点はこちらにお問い合わせください。\n</content>\n<source>\n3\n</source>\n</search_result>\n<search_result>\n<content>\n入社 2〜4 週間目  ### 4.1 詳細な製品トレーニング  - [ ] 各製品・サービスの技術的詳細の学習 - [ ] 製品利用事例とケーススタディの理解 - [ ] 製品デモンストレーションの実践  ### 4.2 営業スキルトレーニング  - [ ] 効果的な電話営業テクニックの学習 - [ ] 商談ロールプレイングセッション - [ ] 提案書作成ワークショップ  ### 4.3 顧客管理実践  - [ ] CRM システムでの顧客情報管理演習 - [ ] リードスコアリングの方法論理解 - [ ] 顧客セグメンテーション演習  ### 4.4 コンプライアンストレーニング  - [ ] 営業倫理に関するオンラインコース受講 - [ ] 個人情報保護法の理解 - [ ] 反社会的勢力対応マニュアルの確認  ## 5. 入社 1〜3 ヶ月目  ### 5.1 実践的営業活動  - [ ] メンター同行での顧客訪問（週 1-2 回） - [ ] 電話でのリードフォロー（日々の業務） - [ ] 小規模案件の単独担当  ### 5.2 提案・見積スキルの向上  - [ ] 提案書作成演習（週 1 回） - [ ] 見積作成システムの詳細トレーニング - [ ] プレゼンテーションスキル向上ワークショップ  ### 5.3 業界知識の深化  - [ ] 業界専門誌の定期購読開始 - [ ] 競合他社の製品比較分析 - [ ] 業界セミナーやウェビナーへの参加  ### 5.4 社内ネットワーキング  - [ ] 他部門（製品開発、マーケティング等）との交流会 - [ ] 経営陣との懇談会 - [ ] 社内メンターシッププログラムへの参加  ## 6.\n</content>\n<source>\n4\n</source>\n</search_result>\n<search_result>\n<content>\n# オフィス利用ガイドライン  1. 一般的なルール     - オフィスの開閉時間を遵守してください（例：平日 8:00-20:00）    - 身分証明書やセキュリティカードを常に携帯し、必要に応じて提示してください    - オフィス内では適切な服装を心がけてください    - 喫煙は指定された場所でのみ行ってください  2. セキュリティ     - 部外者を無断でオフィスに入れないでください    - 機密情報の取り扱いには十分注意し、不要な書類は適切に廃棄してください    - パソコンから離れる際は必ずロックしてください    - セキュリティに関する懸念事項は速やかに報告してください  3. 共有スペース     - 会議室やミーティングスペースは予約システムを利用し、使用後は整理整頓してください    - キッチンや休憩スペースは清潔に保ち、使用後は片付けてください    - 騒音に配慮し、他の従業員の作業を妨げないようにしてください  4. 設備・備品の利用     - オフィス設備や備品は丁寧に扱い、故障や不具合は速やかに報告してください    - 個人的な目的での過度な設備利用（プリンター等）は控えてください    - エネルギー節約のため、不要な照明や機器の電源はこまめに切ってください  5. 健康・安全     - オフィス内での事故や怪我は直ちに報告してください    - 緊急時の避難経路と集合場所を把握しておいてください    - 体調不良時は無理せず、適切な対応（在宅勤務、休暇取得等）を取ってください  6. コミュニケーション     - 同僚への敬意を持ち、ハラスメントのない職場環境を維持してください    - オープンスペースでの長時間の私的な会話は控えてください    - 重要な連絡事項は適切な方法（メール、掲示板等）で共有してください  7. リモートワーク     - リモートワーク時も、通常のオフィスワーク同様のプロ意識を持って業務に取り組んでください    - オンライン会議の際は、適切な環境（背景、照明、音声）を整えてください  8. 環境への配慮     - ゴミの分別を徹底し、リサイクルに協力してください    - 紙の使用を最小限に抑え、可能な限りデジタル化を推進してください  9. 来客対応     - 来客の際は礼儀正しく対応し、必要に応じて適切な案内をしてください    - 機密情報が見える場所に来客を案内する際は十分注意してください  10. ガイドラインの遵守     - このガイドラインに違反する行為を見かけた場合は、適切な方法で報告してください     - ガイドラインは定期的に見直され、更新される可能性があります。最新版を常に確認してください  このガイドラインは、快適で生産的なオフィス環境を維持するためのものです。皆様のご協力をお願いいたします。\n</content>\n<source>\n5\n</source>\n</search_result>\n\n\n</search_results>\n\nIf you reference information from a search result within your answer, you must include a citation to source where the information was found. Each result has a corresponding source ID that you should reference.\n\nNote that <sources> may contain multiple <source> if you include information from multiple results in your answer.\n\nDo NOT directly quote the <search_results> in your answer. Your job is to answer the user's question as concisely as possible.\n\nYou must output your answer in the following format. Pay attention and follow the formatting and spacing exactly:\n<answer>\n<answer_part>\n<text>\nfirst answer text\n</text>\n<sources>\n<source>source ID</source>\n</sources>\n</answer_part>\n<answer_part>\n<text>\nsecond answer text\n</text>\n<sources>\n<source>source ID</source>\n</sources>\n</answer_part>\n</answer>",
            "max_tokens": 2048,
            "temperature": 0,
            "top_p": 1,
            "stop_sequences": [
                "\nObservation"
            ],
            "top_k": 50
        },
        "inputTokenCount": 4367
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "id": "msg_bdrk_01DNy6f5ycKEWt3Hm3CPYACn",
            "type": "message",
            "role": "assistant",
            "model": "claude-3-sonnet-20240229",
            "content": [
                {
                    "type": "text",
                    "text": "<answer>\n<answer_part>\n<text>\nオフィスの会議室を借りるには、予約システムを利用する必要があります。会議室の予約は以下の手順で行ってください:\n\n1. 予約システムにログインする\n2. 利用したい会議室と日時を選択する\n3. 予約を確定する\n\n会議室の利用後は、整理整頓して次の利用者のために備えておく必要があります。\n</text>\n<sources>\n<source>5</source>\n</sources>\n</answer_part>\n\n<answer_part>\n<text>\n会議室の予約可能な時間帯は、オフィスの開閉時間に準じています。検索結果によると、オフィスの開閉時間は平日の8:00から20:00までとなっています。したがって、会議室の予約可能な時間帯は平日の8:00から20:00までとなります。\n</text>\n<sources>\n<source>5</source>\n</sources>\n</answer_part>\n</answer>"
                }
            ],
            "stop_reason": "end_turn",
            "stop_sequence": null,
            "usage": {
                "input_tokens": 4367,
                "output_tokens": 298
            }
        },
        "outputTokenCount": 298
    }
}
```

## まとめ

このセクションでは以下のことを学びました。

- クエリ分割ではユーザーからの質問をそのまま RAG に通さず、スタンドアロンクエリとサブクエリに分割する
- スタンドアロンクエリはユーザープロンプトにクエリ書き換えされ、 `messages` 内に保管される
- 各クエリに対してエンべディング、ベクトル検索される
  - クエリの数 ≠ ベクトル検索の結果であった

Next: [後片付け](./07.md)